In [45]:
print("Language classifier")

Language classifier


In [46]:
import pandas as pd

In [58]:
data = pd.read_csv("processedData.csv")
test_data = pd.read_csv("test.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            500 non-null    int64 
 1   message               500 non-null    object
 2   fingers               500 non-null    int64 
 3   tail                  500 non-null    object
 4   species               500 non-null    object
 5   preprocessed_message  500 non-null    object
 6   processedMessage      500 non-null    object
 7   species_group         500 non-null    object
dtypes: int64(2), object(6)
memory usage: 31.4+ KB


In [59]:
data

,Unnamed: 0,message,fingers,tail,species,preprocessed_message,processedMessage,species_group
0,0,pluvia arbor aquos,4,no,Aquari,"['rain', 'a', 'tree', 'water']",rain tree water,Aquari
1,1,cosmix xeno nebuz odbitaz,5,yes,Zorblax,"['cosmos', 'xeno', 'in', 'the', 'sky', 'reject...",cosmos xeno sky rejected,Zorblax
2,2,solarix glixx novum galaxum quasar,5,yes,Zorblax,"['solar', 'glix', 'new', 'galaxy', 'quasar']",solar glix new galaxy quasar,Zorblax
3,3,arbor insectus pesros ekos dootix nimbus,2,yes,Florian,"['a', 'tree', 'insect', 'peso', 'eco', 'gifted...",tree insect peso eco gifted cloud,Florian
4,4,mermax drakos lorix epikoz deftax,4,no,Faerix,"['merma', 'dragon', 'to', 'wear', 'epic', 'deft']",merma dragon wear epic deft,Faerix_Group2
...,...,...,...,...,...,...,...,...
495,495,empathix sadix disgux dredax pridius afgstix e...,2,no,Emotivor,"['empathize', 'sad', 'discus', 'dreds', 'earli...",empathize sad discus dreds earlier afgst excited,Emotivor_Group2
496,496,quasar ustron nebulax meteorn,4,no,Quixnar,"['quasar', 'burn', 'fog', 'meteorn']",quasar burn fog meteorn,Quixnar
497,497,astron xeno ceaestar astron kometa,6,yes,Zorblax,"['astron', 'xeno', 'dinner', 'astron', 'comet']",astron xeno dinner astron comet,Zorblax
498,498,sporzom nimbus terram terranix aviana ekos nimbub,2,yes,Florian,"['sporzom', 'cloud', 'the', 'earth', 'earthlin...",sporzom cloud earth earthling avian eco nimbub,Florian


In [60]:
data['tail']=data['tail'].map({'yes':1, 'no':0})

In [61]:
data

,Unnamed: 0,message,fingers,tail,species,preprocessed_message,processedMessage,species_group
0,0,pluvia arbor aquos,4,0,Aquari,"['rain', 'a', 'tree', 'water']",rain tree water,Aquari
1,1,cosmix xeno nebuz odbitaz,5,1,Zorblax,"['cosmos', 'xeno', 'in', 'the', 'sky', 'reject...",cosmos xeno sky rejected,Zorblax
2,2,solarix glixx novum galaxum quasar,5,1,Zorblax,"['solar', 'glix', 'new', 'galaxy', 'quasar']",solar glix new galaxy quasar,Zorblax
3,3,arbor insectus pesros ekos dootix nimbus,2,1,Florian,"['a', 'tree', 'insect', 'peso', 'eco', 'gifted...",tree insect peso eco gifted cloud,Florian
4,4,mermax drakos lorix epikoz deftax,4,0,Faerix,"['merma', 'dragon', 'to', 'wear', 'epic', 'deft']",merma dragon wear epic deft,Faerix_Group2
...,...,...,...,...,...,...,...,...
495,495,empathix sadix disgux dredax pridius afgstix e...,2,0,Emotivor,"['empathize', 'sad', 'discus', 'dreds', 'earli...",empathize sad discus dreds earlier afgst excited,Emotivor_Group2
496,496,quasar ustron nebulax meteorn,4,0,Quixnar,"['quasar', 'burn', 'fog', 'meteorn']",quasar burn fog meteorn,Quixnar
497,497,astron xeno ceaestar astron kometa,6,1,Zorblax,"['astron', 'xeno', 'dinner', 'astron', 'comet']",astron xeno dinner astron comet,Zorblax
498,498,sporzom nimbus terram terranix aviana ekos nimbub,2,1,Florian,"['sporzom', 'cloud', 'the', 'earth', 'earthlin...",sporzom cloud earth earthling avian eco nimbub,Florian


In [62]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
data['species_number'] = label_encoder.fit_transform(data['species_group'])

In [63]:
data

,Unnamed: 0,message,fingers,tail,species,preprocessed_message,processedMessage,species_group,species_number
0,0,pluvia arbor aquos,4,0,Aquari,"['rain', 'a', 'tree', 'water']",rain tree water,Aquari,0
1,1,cosmix xeno nebuz odbitaz,5,1,Zorblax,"['cosmos', 'xeno', 'in', 'the', 'sky', 'reject...",cosmos xeno sky rejected,Zorblax,11
2,2,solarix glixx novum galaxum quasar,5,1,Zorblax,"['solar', 'glix', 'new', 'galaxy', 'quasar']",solar glix new galaxy quasar,Zorblax,11
3,3,arbor insectus pesros ekos dootix nimbus,2,1,Florian,"['a', 'tree', 'insect', 'peso', 'eco', 'gifted...",tree insect peso eco gifted cloud,Florian,6
4,4,mermax drakos lorix epikoz deftax,4,0,Faerix,"['merma', 'dragon', 'to', 'wear', 'epic', 'deft']",merma dragon wear epic deft,Faerix_Group2,5
...,...,...,...,...,...,...,...,...,...
495,495,empathix sadix disgux dredax pridius afgstix e...,2,0,Emotivor,"['empathize', 'sad', 'discus', 'dreds', 'earli...",empathize sad discus dreds earlier afgst excited,Emotivor_Group2,3
496,496,quasar ustron nebulax meteorn,4,0,Quixnar,"['quasar', 'burn', 'fog', 'meteorn']",quasar burn fog meteorn,Quixnar,9
497,497,astron xeno ceaestar astron kometa,6,1,Zorblax,"['astron', 'xeno', 'dinner', 'astron', 'comet']",astron xeno dinner astron comet,Zorblax,11
498,498,sporzom nimbus terram terranix aviana ekos nimbub,2,1,Florian,"['sporzom', 'cloud', 'the', 'earth', 'earthlin...",sporzom cloud earth earthling avian eco nimbub,Florian,6


In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_message = vectorizer.fit_transform(data['processedMessage'])

In [67]:
X_message.shape

(500, 525)

In [68]:
X_message[:5].todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [69]:
feature_names = vectorizer.get_feature_names_out()
print(feature_names[:40]) 

['acon' 'aeon' 'aeop' 'afgst' 'afraid' 'air' 'akiana' 'akon' 'algorithm'
 'alone' 'angsqi' 'anxiety' 'anxious' 'anxiously' 'aobla' 'aqjos' 'aqpos'
 'archer' 'around' 'aryan' 'asbron' 'aslron' 'astrln' 'astroh' 'astron'
 'astrrn' 'avcana' 'avian' 'awesome' 'awezo' 'awezon' 'awozom' 'bark'
 'beaded' 'beast' 'beepod' 'bgissa' 'bioke' 'biomar' 'biompl']


In [70]:
first_message_tfidf = X_message[0].todense().tolist()[0] 
word_tfidf_pairs = [(feature_names[i], first_message_tfidf[i]) for i in range(len(feature_names)) if first_message_tfidf[i] > 0]
print(word_tfidf_pairs)

[('rain', 0.5589095731736434), ('tree', 0.5787706375476005), ('water', 0.5938388991364575)]


In [71]:
sparsity = 1.0 - (X_message.count_nonzero() / float(X_message.shape[0] * X_message.shape[1]))
print(f"Sparsity: {sparsity * 100:.2f}%")

Sparsity: 99.04%


In [72]:
import numpy as np
from scipy.sparse import hstack

X_features = hstack([X_message, np.array(data[['fingers', 'tail']])])
y = data['species']

In [84]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
def combine_labels(y):
    return y.replace({
        'Faerix_Group1': 'Faerix',
        'Faerix_Group2': 'Faerix',
        'Emotivor_Group1': 'Emotivor',
        'Emotivor_Group2': 'Emotivor'
    })



X_train, X_val, y_train, y_val = train_test_split(X_features, y, test_size=0.15, random_state=12)

model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_val)

# Combine the labels for both predicted and actual values
y_pred_combined = combine_labels(pd.Series(y_pred))
y_val_combined = combine_labels(pd.Series(y_val))

# Calculate the accuracy based on the combined labels
accuracy = accuracy_score(y_val_combined, y_pred_combined)
print(f'Validation Accuracy: {accuracy}')

# pro_sum = 0.0
# ind = -1
# max = 0.0
# for i in range(1, 41):
#     X_train, X_val, y_train, y_val = train_test_split(X_features, y, test_size=0.15, random_state=i)

#     model = RandomForestClassifier()
#     model.fit(X_train, y_train)

#     y_pred = model.predict(X_val)

#     # Combine the labels for both predicted and actual values
#     y_pred_combined = combine_labels(pd.Series(y_pred))
#     y_val_combined = combine_labels(pd.Series(y_val))

#     accuracy = accuracy_score(y_val_combined, y_pred_combined)
#     # if accuracy > max:
#     #     max = accuracy
#     #     ind = i
#     pro_sum += accuracy
# print(f'Average Validation Accuracy: {pro_sum / 40}')

Validation Accuracy: 0.88


In [28]:
test_data['tail']=test_data['tail'].map({'yes':1, 'no':0})

In [81]:
test_data

,message,fingers,tail
0,iephyr terram nimbus terram faunar foliar,2,0
1,joyzor uleex luvium caloox shockus blissae,4,1
2,aquos arbor ventuc,4,1
3,nympha nympha epikoz nympha mythox mythox mythox,3,0
4,diitax sibenix fabulon,4,1
...,...,...,...
294,fabuion drakos lorix relikum cyclopix,4,1
295,codex cybrex algorix synapz mechan nanozom dotax,5,1
296,centarex mythox nympha krakos,3,1
297,orbitaz astron glixx novara novrm ufox qcasar ...,3,0


In [82]:
X_test_message = vectorizer.transform(test_data['message'])
X_test_features = hstack([X_test_message, np.array(test_data[['fingers', 'tail']])])

In [83]:
y_test_pred = model.predict(X_test_features)

In [88]:
y_test_pred

array(['Aquari', 'Sentire', 'Florian', 'Mythron', 'Mythron', 'Faerix',
       'Faerix', 'Faerix', 'Emotivor', 'Florian', 'Mythron', 'Emotivor',
       'Zorblax', 'Quixnar', 'Mythron', 'Aquari', 'Florian', 'Cybex',
       'Aquari', 'Aquari', 'Quixnar', 'Emotivor', 'Quixnar', 'Nexoon',
       'Mythron', 'Nexoon', 'Aquari', 'Sentire', 'Emotivor', 'Mythron',
       'Cybex', 'Emotivor', 'Mythron', 'Emotivor', 'Mythron', 'Emotivor',
       'Aquari', 'Florian', 'Aquari', 'Quixnar', 'Emotivor', 'Quixnar',
       'Mythron', 'Nexoon', 'Aquari', 'Emotivor', 'Quixnar', 'Quixnar',
       'Zorblax', 'Florian', 'Emotivor', 'Quixnar', 'Sentire', 'Quixnar',
       'Emotivor', 'Emotivor', 'Sentire', 'Emotivor', 'Mythron', 'Cybex',
       'Cybex', 'Cybex', 'Mythron', 'Emotivor', 'Sentire', 'Mythron',
       'Sentire', 'Mythron', 'Nexoon', 'Quixnar', 'Quixnar', 'Emotivor',
       'Emotivor', 'Zorblax', 'Emotivor', 'Sentire', 'Cybex', 'Faerix',
       'Nexoon', 'Emotivor', 'Emotivor', 'Cybex', 'Emotivor', 

In [93]:
pd.DataFrame(y_test_pred)

,0
0,Aquari
1,Sentire
2,Florian
3,Mythron
4,Mythron
...,...
294,Mythron
295,Nexoon
296,Mythron
297,Quixnar
